[View in Colaboratory](https://colab.research.google.com/github/bixbyr/LeelaZero_Analysis/blob/master/Leela_Zero_Progress.ipynb)

In [1]:
!pip install -q git+https://github.com/altair-viz/altair
import altair as alt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib3
from datetime import datetime

pd.options.mode.chained_assignment = None
alt.renderers.enable('colab')

RendererRegistry.enable('colab')

In [31]:
http = urllib3.PoolManager()
soup = BeautifulSoup(http.request('GET', 'http://zero.sjeng.org/').data, 'html.parser')

table = soup.body.table
columns = [c.string for c in table.contents[2].find_all("th")]
print(columns)

row_data = []
for r in table.find_all('tr'):
  row_contents = [c.string for c in r.find_all("td")]
  if len(row_contents) > 0:
    row_data.append(row_contents)

df = pd.DataFrame.from_records(row_data, columns=columns)
df["#"] = pd.to_numeric(df["#"])
df["Games"] = pd.to_numeric(df["Games"])
df["Elo"] = pd.to_numeric(df["Elo"])
df.rename(columns={"#": "Network #", "Size": "Network"}, inplace=True)
df["Upload Date"] = pd.to_datetime(df["Upload Date"])

# Assume that from 2018-05-09 onwards 25% of all played games were LeelaElf games
elf_games = df[df["Upload Date"] >= datetime(2018,5,9,0,0)]
elf_games["Network"] = "LeelaElf"
elf_games["Games"] = (elf_games["Games"] * 1./3.).round()
df = df.append(elf_games)

df.sort_values(["Network #", "Upload Date"], inplace=True)
df["Days"] = ((df["Upload Date"] - df["Upload Date"].min()) / np.timedelta64(1, 'D')).apply(np.floor)
df["Week"] = (df["Days"] / 7).apply(np.floor)

day_stat = df.groupby(["Days", "Network"]).agg({"Games": "sum", "Elo": "mean"})
week_stat = df.groupby(["Week", "Network"])[["Games"]].sum()

['#', 'Upload Date', 'Hash', 'Size', 'Elo', 'Games', 'Training #']


In [32]:
alt.Chart(df).mark_bar().encode(
    x="Network #",
    y="Games",
    color="Network"
).properties(
    width=800,
    height=600,
    title="Games per network"
)

Chart({
  data:      Network #         Upload Date      Hash   Network    Elo     Games  \
  145          0 2017-11-10 13:04:00  d645af97       8x1      0   15036.0   
  144          1 2017-11-15 17:07:00  e6779c9b       8x1    164   23639.0   
  143          2 2017-11-17 08:09:00  fe3f6afd      32x4     91   44001.0   
  142          3 2017-11-19 11:44:00  4d9a5cfc      32x4    477   76711.0   
  141          4 2017-11-21 08:06:00  857b975e      32x4    532   15178.0   
  140          5 2017-11-21 14:37:00  1ac2638d      64x5    542  104810.0   
  139          6 2017-11-24 16:08:00  a1151640      64x5    892   36363.0   
  138          7 2017-11-25 13:16:00  92c658d7      64x5    917  192872.0   
  137          8 2017-11-30 22:25:00  22373747      64x5   1282  111578.0   
  136          9 2017-12-03 22:46:00  6f274ab1      64x5   1523  137868.0   
  135         10 2017-12-06 09:42:00  ad8a1d02      64x5   1650   75393.0   
  134         11 2017-12-07 13:07:00  2184b750      64x5   1665   68977.0   
  133         12 2017-12-09 08:22:00  eebb910d      64x5   1732   18400.0   
  132         13 2017-12-09 19:13:00  1607dbea      64x5   1927   28131.0   
  131         14 2017-12-11 06:43:00  96bf20e7      64x5   2255    7192.0   
  130         15 2017-12-11 12:07:00  a0d3531f      64x5   2381    9515.0   
  129         16 2017-12-11 18:59:00  fe7a6c28      64x5   2604   20389.0   
  128         17 2017-12-12 06:33:00  5bcb392e      64x5   2893   38353.0   
  127         18 2017-12-13 06:21:00  4701bb9f      64x5   2923   24559.0   
  126         19 2017-12-14 15:48:00  e671af6f      64x5   3077    7939.0   
  125         20 2017-12-14 21:19:00  251b7ca8      64x5   3197   27205.0   
  124         21 2017-12-15 16:46:00  d11131e3      64x5   3305    6650.0   
  123         22 2017-12-15 21:09:00  3ba92804      64x5   3425   16320.0   
  122         23 2017-12-16 06:25:00  4e4d09be      64x5   3523    7009.0   
  121         24 2017-12-16 16:46:00  3382a7d3      64x5   3726   34833.0   
  120         25 2017-12-17 15:54:00  808ac5f3      64x5   3808   15947.0   
  119         26 2017-12-18 07:50:00  0eb8b23b      64x5   3964   23810.0   
  118         27 2017-12-18 23:44:00  ce1aa643      64x5   4010   17064.0   
  117         28 2017-12-19 14:18:00  154eaebd      64x5   4168   12493.0   
  116         29 2017-12-19 22:29:00  657f91c7      64x5   4244   34864.0   
  ..         ...                 ...       ...       ...    ...       ...   
  19         126 2018-04-21 00:11:00  59bb7337    192x15  10548   10886.0   
  18         127 2018-04-22 14:49:00  3f6c8dd8    192x15  10574   49315.0   
  17         128 2018-04-25 07:23:00  158603eb    192x15  10604   62391.0   
  16         129 2018-04-29 09:00:00  cfb2ce28    192x15  10655   53716.0   
  15         130 2018-05-02 03:06:00  18e6a6c5    192x15  10710   61580.0   
  14         131 2018-05-04 07:45:00  ecab83bb    192x15  10745   31498.0   
  13         132 2018-05-06 20:26:00  3737d7ce    192x15  10808    3248.0   
  12         133 2018-05-07 06:38:00  4be6f310    192x15  10844    1496.0   
  11         134 2018-05-07 11:10:00  2fb02cfe    192x15  10880    4544.0   
  10         135 2018-05-08 00:50:00  05b723d0    192x15  10924   10769.0   
  9          136 2018-05-09 16:04:00  b6337c69    192x15  10959   50480.0   
  9          136 2018-05-09 16:04:00  b6337c69  LeelaElf  10959   16827.0   
  8          137 2018-05-14 08:38:00  90560d92    192x15  10992   31532.0   
  8          137 2018-05-14 08:38:00  90560d92  LeelaElf  10992   10511.0   
  7          138 2018-05-17 02:39:00  d8214630    192x15  11043    8822.0   
  7          138 2018-05-17 02:39:00  d8214630  LeelaElf  11043    2941.0   
  6          139 2018-05-17 20:50:00  a39b64cc    192x15  11095   28474.0   
  6          139 2018-05-17 20:50:00  a39b64cc  LeelaElf  11095    9491.0   
  5          140 2018-05-20 01:21:00  521b0868    192x15  11142    5553.0   
  5          140 2018-05-20 01:21:00  521b0868  LeelaElf  1

In [36]:
alt.Chart(week_stat.reset_index()).mark_bar().encode(
    x="Week",
    y="Games",
    color="Network"
).properties(
#     width=800,
#     height=600,
    title="Games per week"
)

Chart({
  data:     Week   Network     Games
  0    0.0      32x4   44001.0
  1    0.0       8x1   38675.0
  2    1.0      32x4   91889.0
  3    1.0      64x5  104810.0
  4    2.0      64x5  340813.0
  5    3.0      64x5  282238.0
  6    4.0      64x5  181683.0
  7    5.0      64x5  225325.0
  8    6.0      64x5  215044.0
  9    7.0      64x5  276038.0
  10   8.0      64x5  671031.0
  11   9.0      64x5  514488.0
  12  10.0     128x6  233048.0
  13  11.0     128x6  249199.0
  14  12.0     128x6  284508.0
  15  13.0     128x6  206156.0
  16  14.0     128x6  394433.0
  17  15.0     128x6  323171.0
  18  16.0    128x10  200629.0
  19  16.0     128x6  188562.0
  20  17.0    128x10  689913.0
  21  18.0    128x10  288606.0
  22  19.0    128x10  228331.0
  23  20.0    128x10  437810.0
  24  21.0    192x15   73142.0
  25  22.0    192x15  212315.0
  26  23.0    192x15  122592.0
  27  24.0    192x15  146794.0
  28  25.0    192x15   70537.0
  29  25.0  LeelaElf   16827.0
  30  26.0    192x15   68828.0
  31  26.0  LeelaElf   22943.0
  32  27.0    192x15  141895.0
  33  27.0  LeelaElf   47298.0
  34  28.0    192x15   13501.0
  35  28.0  LeelaElf    4500.0,
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'Network'
    }),
    x: X({
      shorthand: 'Week'
    }),
    y: Y({
      shorthand: 'Games'
    })
  }),
  mark: 'bar',
  title: 'Games per week'
})

In [34]:
df_noelf = df[df["Network"] != "LeelaElf"]
line_plot = alt.Chart(df_noelf).mark_line().encode(
    x="Days",
    y="Elo",
    color="Network"
).properties(
     width=800,
     height=600,
    title="Strength over time"
)

elf = df_noelf.copy()
elf["Elo"] = 11560
elf["Network"] = "Measured LeelaElf"

elf_guess = df_noelf.copy()
elf_guess["Elo"] = (11560-11185)*3 + 11185
elf_guess["Network"] = "Estimated LeelaElf"

elf_line = alt.Chart(elf).mark_rule().encode(
    y='Elo',
    color='Network'
    #size=alt.value(3)
)
elf_guess_line = alt.Chart(elf_guess).mark_rule().encode(
    y='Elo',
    color='Network'
)

line_plot + elf_line + elf_guess_line

LayerChart({
  layer: [Chart({
    data:      Network #         Upload Date      Hash Network    Elo     Games  \
    145          0 2017-11-10 13:04:00  d645af97     8x1      0   15036.0   
    144          1 2017-11-15 17:07:00  e6779c9b     8x1    164   23639.0   
    143          2 2017-11-17 08:09:00  fe3f6afd    32x4     91   44001.0   
    142          3 2017-11-19 11:44:00  4d9a5cfc    32x4    477   76711.0   
    141          4 2017-11-21 08:06:00  857b975e    32x4    532   15178.0   
    140          5 2017-11-21 14:37:00  1ac2638d    64x5    542  104810.0   
    139          6 2017-11-24 16:08:00  a1151640    64x5    892   36363.0   
    138          7 2017-11-25 13:16:00  92c658d7    64x5    917  192872.0   
    137          8 2017-11-30 22:25:00  22373747    64x5   1282  111578.0   
    136          9 2017-12-03 22:46:00  6f274ab1    64x5   1523  137868.0   
    135         10 2017-12-06 09:42:00  ad8a1d02    64x5   1650   75393.0   
    134         11 2017-12-07 13:07:00  2184b750    64x5   1665   68977.0   
    133         12 2017-12-09 08:22:00  eebb910d    64x5   1732   18400.0   
    132         13 2017-12-09 19:13:00  1607dbea    64x5   1927   28131.0   
    131         14 2017-12-11 06:43:00  96bf20e7    64x5   2255    7192.0   
    130         15 2017-12-11 12:07:00  a0d3531f    64x5   2381    9515.0   
    129         16 2017-12-11 18:59:00  fe7a6c28    64x5   2604   20389.0   
    128         17 2017-12-12 06:33:00  5bcb392e    64x5   2893   38353.0   
    127         18 2017-12-13 06:21:00  4701bb9f    64x5   2923   24559.0   
    126         19 2017-12-14 15:48:00  e671af6f    64x5   3077    7939.0   
    125         20 2017-12-14 21:19:00  251b7ca8    64x5   3197   27205.0   
    124         21 2017-12-15 16:46:00  d11131e3    64x5   3305    6650.0   
    123         22 2017-12-15 21:09:00  3ba92804    64x5   3425   16320.0   
    122         23 2017-12-16 06:25:00  4e4d09be    64x5   3523    7009.0   
    121         24 2017-12-16 16:46:00  3382a7d3    64x5   3726   34833.0   
    120         25 2017-12-17 15:54:00  808ac5f3    64x5   3808   15947.0   
    119         26 2017-12-18 07:50:00  0eb8b23b    64x5   3964   23810.0   
    118         27 2017-12-18 23:44:00  ce1aa643    64x5   4010   17064.0   
    117         28 2017-12-19 14:18:00  154eaebd    64x5   4168   12493.0   
    116         29 2017-12-19 22:29:00  657f91c7    64x5   4244   34864.0   
    ..         ...                 ...       ...     ...    ...       ...   
    29         116 2018-04-04 09:37:00  39d46507  128x10   9888  225231.0   
    28         117 2018-04-09 10:35:00  ba748d40  192x15  10214    7851.0   
    27         118 2018-04-09 18:54:00  02e663b0  192x15  10244    4338.0   
    26         119 2018-04-09 21:12:00  5838db67  192x15  10281    6320.0   
    25         120 2018-04-10 10:50:00  8c67ecdc  192x15  10318   21462.0   
    24         121 2018-04-12 01:36:00  5d6d9c5b  192x15  10343   33171.0   
    23         122 2018-04-13 22:02:00  c0cb605b  192x15  10394   16730.0   
    22         123 2018-04-14 11:38:00  f3231682  192x15  10425   64768.0   
    21         124 2018-04-16 22:11:00  8a045bce  192x15  10487   39905.0   
    20         125 2018-04-18 04:43:00  e8601c38  192x15  10511   90912.0   
    19         126 2018-04-21 00:11:00  59bb7337  192x15  10548   10886.0   
    18         127 2018-04-22 14:49:00  3f6c8dd8  192x15  10574   49315.0   
    17         128 2018-04-25 07:23:00  158603eb  192x15  10604   62391.0   
    16         129 2018-04-29 09:00:00  cfb2ce28  192x15  10655   53716.0   
    15         130 2018-05-02 03:06:00  18e6a6c5  192x15  10710   61580.0   
    14         131 2018-05-04 07:45:00  ecab83bb  192x15  10745   31498.0   
    13         132 2018-05-06 20:26:00  3737d7ce  192x15  10808    3248.0   
    12         133 2018-05-07 06:38:00  4be6f310  192x15  10844    1496.0   
    11         134 2018-05-07 11:10:00  2fb02cfe  192x15  10880    4544.0   
    10         135 2018-05-08 00:50:

,Network #,Upload Date,Hash,Network,Elo,Games,Training #,Days,Week,elf_fraction
145,0,2017-11-10 13:04:00,d645af97,8x1,0,15036.0,0,0.0,0.0,0
144,1,2017-11-15 17:07:00,e6779c9b,8x1,164,23639.0,9000,5.0,0.0,0
143,2,2017-11-17 08:09:00,fe3f6afd,32x4,91,44001.0,19000,6.0,0.0,0
142,3,2017-11-19 11:44:00,4d9a5cfc,32x4,477,76711.0,62000,8.0,1.0,0
141,4,2017-11-21 08:06:00,857b975e,32x4,532,15178.0,107000,10.0,1.0,0
140,5,2017-11-21 14:37:00,1ac2638d,64x5,542,104810.0,137000,11.0,1.0,0
139,6,2017-11-24 16:08:00,a1151640,64x5,892,36363.0,265000,14.0,2.0,0
138,7,2017-11-25 13:16:00,92c658d7,64x5,917,192872.0,292000,15.0,2.0,0
137,8,2017-11-30 22:25:00,22373747,64x5,1282,111578.0,485000,20.0,2.0,0
136,9,2017-12-03 22:46:00,6f274ab1,64x5,1523,137868.0,585000,23.0,3.0,0
